In [1]:
import pandas as pd
import json
import random
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

In [7]:
def get_no_emo_data(file_path):
    chats = []
    for line in open(file_path, 'r'):
        chats.append(json.loads(line))
    data = []
    for chat in chats:
        for text in chat['dialogue']:
            if text['emotion'] == 'no_emotion':
                data.append(text['text'])
    return data

def get_empathy_data(file_path):
    data = pd.read_csv(file_path,error_bad_lines=False)
    data = data[['conv_id','prompt','context']].groupby("conv_id").agg('max')
    #data = list(map(lambda x: x.replace('_comma_',' ,'),data))
    return data
            

In [8]:
train_path = '../data/dailydialog/train.json'
val_path = '../data/dailydialog/valid.json'
test_path = '../data/dailydialog/test.json'
no_emo_data = get_no_emo_data(train_path) + get_no_emo_data(val_path) + get_no_emo_data(test_path)

In [3]:
empathy_train_path = "../data/empatheticdialogues/train.csv"
empathy_valid_path = "../data/empatheticdialogues/valid.csv"
empathy_test_path = "../data/empatheticdialogues/test.csv"
#empathy_data = get_empathy_data(empathy_train_path)+get_empathy_data(empathy_valid_path)+get_empathy_data(empathy_test_path)

In [4]:
x = pd.read_csv(empathy_train_path,error_bad_lines=False)

b'Skipping line 2355: expected 8 fields, saw 10\nSkipping line 36628: expected 8 fields, saw 12\nSkipping line 49433: expected 8 fields, saw 10\nSkipping line 56957: expected 8 fields, saw 10\nSkipping line 65019: expected 8 fields, saw 10\n'


In [6]:
x.head(20)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN
5,hit:0_conv:1,6,sentimental,I remember going to the fireworks with my best...,0,Oh was this something that happened because of...,5|5|5_2|2|5,NaN
6,hit:1_conv:2,1,afraid,i used to scare for darkness,2,it feels like hitting to blank wall when i se...,4|3|4_3|5|5,NaN
7,hit:1_conv:2,2,afraid,i used to scare for darkness,3,Oh ya? I don't really see how,4|3|4_3|5|5,NaN
8,hit:1_conv:2,3,afraid,i used to scare for darkness,2,dont you feel so.. its a wonder,4|3|4_3|5|5,NaN
9,hit:1_conv:2,4,afraid,i used to scare for darkness,3,I do actually hit blank walls a lot of times b...,4|3|4_3|5|5,NaN


In [7]:
no_emo_data_sampled = random.sample(no_emo_data,len(empathy_data))

In [8]:
no_emo_label = [0]*len(empathy_data)
empathy_label = [1]*len(empathy_data)

In [9]:
data = no_emo_data_sampled + empathy_data
label = no_emo_label + empathy_label

In [23]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.15, random_state=42)

In [22]:
len(x_train)*0.9

33327.0

In [24]:
pickle.dump(x_train,open( "../data/x_train.p", "wb" ))
pickle.dump(y_train,open( "../data/y_train.p", "wb" ))
pickle.dump(x_test,open( "../data/x_test.p", "wb" ))
pickle.dump(y_test,open( "../data/y_test.p", "wb" ))

In [9]:
x = pickle.load(open( "../data/x_train.p", "rb" ))
y = pickle.load(open( "../data/y_train.p", "rb" ))

In [11]:
y

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


In [8]:
x = pickle.load(open( "../data/tpchat/dialoguedata.p", "rb" ))

[[314, 4240, 611, 484, 1138, 703, 326, 561, 467, 422, 612],
 [1312, 4724, 29294, 810, 262, 9546, 366, 9246, 25422, 1, 2058, 422],
 [3363,
  326,
  318,
  4998,
  13,
  679,
  1839,
  262,
  9764,
  9071,
  17792,
  12,
  11693,
  12184,
  736,
  287,
  24648,
  287,
  1109,
  13],
 [314, 4240, 611, 484, 550, 1138, 644, 339, 561, 423, 3194, 546, 607, 13],
 [314, 760, 783, 790, 2435, 314, 1280, 308, 4529, 314, 892, 286, 5482, 3245]]

In [ ]:
ed = pd.read_csv(empathy_valid_path,error_bad_lines=False)

In [20]:
ed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76668 entries, 0 to 76667
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   conv_id        76668 non-null  object
 1   utterance_idx  76668 non-null  int64 
 2   context        76668 non-null  object
 3   prompt         76668 non-null  object
 4   speaker_idx    76668 non-null  int64 
 5   utterance      76668 non-null  object
 6   selfeval       76668 non-null  object
 7   tags           693 non-null    object
dtypes: int64(2), object(6)
memory usage: 4.7+ MB


In [15]:
df = ed[['conv_id','context','prompt']].groupby('conv_id').agg('max')

In [17]:
pickle.dump(df,open( "../data/empathy_cls_valid_df.p", "wb" ))

In [16]:
df

,context,prompt
conv_id,,
hit:10001_conv:20003,annoyed,My upstairs neighbors make a ton of noise at a...
hit:10003_conv:20007,excited,I got my bonus coming up on this paycheck. I h...
hit:10004_conv:20009,confident,I was sure I would do good on this exam coming...
hit:10007_conv:20015,anxious,I have a very hard time taking tests. When I k...
hit:1000_conv:2000,faithful,My coworker came on to me at work last week_co...
...,...,...
hit:9992_conv:19985,devastated,My neice and I were distraught because she wor...
hit:9993_conv:19987,anxious,I will be going to New York Next week. I cant ...
hit:9997_conv:19994,hopeful,My coworkers and I bought 200 dollars worth of...


In [2]:
!pip install beepy

     |████████████████████████████████| 1.3 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 16.0 MB/s eta 0:00:01
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp38-cp38-linux_x86_64.whl size=2053452 sha256=d94bc7b9d61e511763666e1910caea207186e2e9de3e569a0761e823f04b6e3b
  Stored in directory: /local/scratch/gaurav/proj/.cache/pip/wheels/3c/46/0a/194e1db8ba343c98e075410186eba0b3bce3993062cdb0169d
Successfully built simpleaudio


In [4]:
import beepy

ModuleNotFoundError: No module named 'beepy'

In [ ]:
beep(sound=1) # integer as argument
beep(sound='coin') # string as argument

In [6]:
2736/16

171.0

In [27]:
x = np.array(pickle.load(open("../data/x_train.p","rb")))
y = np.array(pickle.load(open("../data/y_train.p","rb")))
no_ques = []
ques = []
x0 = x[y==0]
for text in x0:
    if text[-1]!='?':
        no_ques.append(text)
    else:
        ques.append(text)

In [28]:
len(ques)

7017

In [29]:
len(no_ques)

12587

In [55]:
no_emo = np.random.choice(ques[0:4000],600).tolist() + np.random.choice(no_ques[0:7000],1100).tolist()

In [56]:
np.random.shuffle(no_emo)

In [57]:
df = pickle.load(open("../data/empathy_cls_df.p","rb"))
df_val = pickle.load(open("../data/empathy_cls_valid_df.p","rb"))

In [58]:
df.reset_index()[['context','prompt']]

,context,prompt
0,sentimental,I remember going to the fireworks with my best...
1,surprised,My girlfriend got me a toad today! I was so sh...
2,impressed,I really like the new paint job on my house.
3,lonely,I went to the skating rink all by myself today...
4,ashamed,I was walking on the road. I saw beggar and i ...
...,...,...
17834,apprehensive,I went skydiving for the first time recently. ...
17835,confident,I recently took the Law School Admissions Test.
17836,devastated,I was shocked when Lebron left the Cavs again
17837,ashamed,I am ashamed when I cheated on the test.


In [59]:
nedf=pd.DataFrame({
    'context':['no_emo']*len(no_emo),
    'prompt': no_emo
})
df = df.append(nedf)
df

,context,prompt
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...
hit:10000_conv:20000,surprised,My girlfriend got me a toad today! I was so sh...
hit:10000_conv:20001,impressed,I really like the new paint job on my house.
hit:10001_conv:20002,lonely,I went to the skating rink all by myself today...
hit:10002_conv:20004,ashamed,I was walking on the road. I saw beggar and i ...
...,...,...
1695,no_emo,that ' s why ambition need to be realistic . Y...
1696,no_emo,"No , I am afraid we don't ."
1697,no_emo,Here are the plans for the new finished goods ...
1698,no_emo,"I didn ' t see anything out of the ordinary , ..."


In [60]:
pickle.dump(df,open("../data/empathy_cls_ne_df.p","wb"))